In [1]:
@file:DependsOn("/data/shared/antlr-4.9.1-complete.jar")
@file:DependsOn(".")

In [2]:
import org.antlr.v4.runtime.*
import calculator.*

## Work Unit 1

Complete the following visitor implementation

In [6]:
class Visitor: CalcBaseVisitor<Int>() {
    val scope = mutableMapOf<String, Int>()
    override fun visitProgram(ctx: CalcParser.ProgramContext): Int {
        ctx.statement().forEach {
            this.visit(it)
        }
        return 0
    }
    override fun visitAssignStatement(ctx: CalcParser.AssignStatementContext): Int = this.visit(ctx.assign())
    override fun visitExprStatement(ctx: CalcParser.ExprStatementContext): Int {
        val result = this.visit(ctx.expr())
        println("> " + result)
        return result
    }
    override fun visitAssign(ctx: CalcParser.AssignContext): Int {
        val name = ctx.ID().text
        var value = this.visit(ctx.expr())
        scope[name] = value
        
        return value
    }
    override fun visitAdd(ctx: CalcParser.AddContext): Int = this.visit(ctx.e1) + this.visit(ctx.e2)
    override fun visitMul(ctx: CalcParser.MulContext): Int = this.visit(ctx.e1) * this.visit(ctx.e2)
    override fun visitParen(ctx: CalcParser.ParenContext): Int = this.visit(ctx.expr())
    override fun visitNum(ctx: CalcParser.NumContext): Int = ctx.NUM().text.toInt()
    override fun visitId(ctx: CalcParser.IdContext): Int {
        return scope.getOrDefault(ctx.ID().text, 0)
    }
}

In [7]:
fun run(source: String) {
    val input = CharStreams.fromString(source)
    val lexer = CalcLexer(input)
    val tokens = CommonTokenStream(lexer)
    val parser = CalcParser(tokens)
    val tree = parser.program()
    
    Visitor().visit(tree)
}

In [8]:
val source = """
x = 2;
y = 4;
1 + 2;        // = 3
(1+2)*3;      // 3*3 = 9
x + y;        // 2+4 = 6
x * (y + y);  // 2 * (4 + 4) = 16
x = (x * x);  // x=2*2=4
x = (x * x);  // x=4*4=16
x = (x * x);  // x=16*16=256
x
x = (x * x);  // x=256*256=65536
x;
"""

run(source)

> 3
> 9
> 6
> 16
> 256
> 65536
